In [7]:
# This file is part of the ADS Parameter Fitting project.
# must be used under ADS integrated Python env (A)
# namely, ..\ADS_install_path\tools\python\python.exe --> Python 3.13.2
# TODO: whole script is run in Jupyter because ADS python ADI only
# supports IPython kernel !!!

# packages to build DIR env
import os, json
# set ads dict: HPEESOF_DIR and home director : HOME
os.environ['HPEESOF_DIR'] = 'D:/ADS/install'
os.environ['HOME'] = 'D:/ADS/dir'

# # packages to multiprocessing
# # add current working directory
# import sys
# cur_path_nn = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting_local/IV_param_regression/NN_training"
# sys.path.append(cur_path_nn)

In [8]:
# packages to import ADS
from keysight.ads import de
from keysight.ads.de import db_uu as db
from keysight.edatoolbox import ads
import keysight.ads.dataset as dataset
from keysight.edatoolbox import util
from pathlib import Path
from IPython.core import getipython

# packages to import data analysis and save
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import h5py

# for NN model inference
from typing import Literal
import subprocess 
import time

In [ ]:
# Model inference : call func in other Python env

def model_inference(python_path:str, 
                     script_dir:str, 
                     code_file_name:str, 
                     model_path:str, 
                     inference_data_path:str, 
                     inference_data_index:str,
                     output_csv_path:str, 
                     single_input_mode:bool,
                     cvae_ena:bool = False,
                     cvae_mode:Literal['rand', 'mean'] = 'mean',
                     num_sampling:int = 1) -> dict:
    
    if cvae_ena: # CVAE model (prability based) inference
        if inference_data_index == 'None':
            # single input with single a or multiple input
            cmd = [
                python_path, code_file_name,
                "--infer-run", model_path,
                "--input-h5",  inference_data_path,
                "--save-csv",  output_csv_path,
                "--sample-mode", cvae_mode,
                "--num-samples", str(num_sampling) if cvae_mode=='rand' else '1'
        ]
        else:
            cmd = [
            python_path, code_file_name,
            "--infer-run", model_path,
            "--input-h5", inference_data_path,
            "--index", inference_data_index,
            "--save-csv", output_csv_path,
            "--sample-mode", cvae_mode,
            "--num-samples", str(num_sampling) if cvae_mode=='rand' else '1'
            ]

    else: # 2 stage DNN (value based) model inference
        if inference_data_index == 'None':
            # single input with single a or multiple input
            cmd = [
                python_path, code_file_name,
                "--data", inference_data_path,
                "--infer-run", model_path,
                "--input-h5",  inference_data_path,
                "--save-csv",  output_csv_path
        ]
        else:
            cmd = [
            python_path, code_file_name,
            "--data", inference_data_path,
            "--infer-run", model_path,
            "--input-h5", inference_data_path,
            "--index", inference_data_index,
            "--save-csv", output_csv_path
            ]

    result = subprocess.run(cmd, cwd=script_dir, capture_output=True, text=True)
    print(result.stdout)

    # transform output from str to var dict
    import pandas as pd

    df = pd.read_csv(output_csv_path)
    if bool(inference_data_index) & single_input_mode:
    # single input
        df_dict = df.to_dict(orient='list')
        var_dict = {key:str(value) for (key, value) in zip(df_dict['param'], df_dict['value'])}
    else:
    # Batch input
        df_dict = df[df['index'] == 1].to_dict(orient='list')
        df_dict.pop('index')
        var_dict = {key:str(df_dict[key][0]) for key in df_dict}
        
    
    return var_dict


def  proxy_model_inference(python_path:str, 
                     script_dir:str, 
                     code_file_name:str, 
                     model_path:str, 
                     inference_data_path:str, 
                     inference_data_index:str,
                     output_npy_path:str) -> np.ndarray:
    
    if inference_data_index == 'None':
        # single input with single a or multiple input
        cmd = [
            python_path, code_file_name,
            "--data", inference_data_path,
            "--infer-proxy-run", model_path,
            "--proxy-input-h5",  inference_data_path,
            "--save-xhat-npy",  output_npy_path
    ]
    else:
        cmd = [
        python_path, code_file_name,
        "--data", inference_data_path,
        "--infer-proxy-run", model_path,
        "--proxy-input-h5", inference_data_path,
        "--proxy-index", inference_data_index,
        "--save-xhat-npy", output_npy_path
        ]

    result = subprocess.run(cmd, cwd=script_dir, capture_output=True, text=True)
    print(result.stdout)



def plot_proxy_error(inference_data_path:str, x_hat_path:str, inference_data_index:str) -> None:
    if inference_data_index != 'None':
        x_hat = np.load(x_hat_path)[0,:]
        x_original = h5py.File(inference_data_path, 'r')
        x_original = x_original['X'][int(inference_data_index), :, :]
        RMSE = np.sqrt(np.mean((x_hat - x_original) ** 2))
        MAE = np.mean(np.abs(x_hat - x_original))
        NMAE = MAE / np.mean(x_original) *100

        Vds_range = np.arange(-3.5, 8.5 + 0.1, 0.1)

        # plot validation result
        # legend_values = np.linspace(1.5, 6.5, 6)  
        legend_values = np.linspace(1, 7, 7)  
        # create colormap
        cmap = plt.cm.winter
        norm = plt.Normalize(vmin=legend_values.min(), vmax=legend_values.max())

        fig, ax = plt.subplots(1, 3, figsize=(15, 8)) 
        for i in range(7):
            color = cmap(norm(legend_values[i]))
            ax[0].plot( Vds_range, x_original[i, :], label=f"VGS={i+1} V", color = color)
            ax[1].plot( Vds_range, x_hat[i, :], label=f"VGS={i+1} V", color = color)
            ax[2].plot( Vds_range, x_hat[i, :] - x_original[i, :], label=f"VGS={i+1} V", color = color)

        ax[0].legend()
        ax[0].grid(True)
        ax[0].set_title('original I-V (generated, with noise)')
        ax[0].set_xlabel("VDS (V)")
        ax[0].set_ylabel("IDS (A)")
        ax[1].grid(True)
        ax[1].legend()
        ax[1].set_title('predicted I-V (proxy g, DNN based)')
        ax[1].set_xlabel("VDS (V)")
        ax[1].set_ylabel("IDS (A)")
        ax[2].grid(True)
        ax[2].set_title(f'error of both I-V \n average MAE = {MAE:.3f} \n nMAE = {NMAE:.3f}%')
        ax[2].set_xlabel("VDS (V)")
        ax[2].set_ylabel("IDS (A)")

        # add color bar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])  
        cbar = fig.colorbar(sm, ax=ax)
        cbar.set_label("VGS (V)")


    else:
        assert ValueError("No index provided, cannot plot proxy error for multiple samples.")
    



def plot_error(inference_data_path:str, data_save_path:str, inference_data_index:str, infer_param_dict: dict, check_list: list[str]):
    # extract data from both pred and validation
    X_orginal = h5py.File(inference_data_path, 'r')
    X_pred = h5py.File(f"{data_save_path}\\validate.h5", 'r')
    if inference_data_index != 'None':
        index = int(inference_data_index) 
        X_orginal_iv = X_orginal['X'][index, :, :]
        X_orginal_y = X_orginal['Y'][index, :, :]
    else:
        if len(X_orginal['X'].shape) == 2:
            X_orginal_iv = X_orginal['X'][:]
        else:
            X_orginal_iv = X_orginal['X'][0,:]


    X_pred_iv = X_pred['X_iv'][:] if X_pred['X_iv'][:].shape[0]==7 else X_pred['X_iv'][0, :, :]
    

    if inference_data_index != 'None':
        # print error of predicted paramters
        print('Relative errors of predicted paramters:')
        for index,key in enumerate(infer_param_dict):
            val = float(infer_param_dict[key])  
            ref = float(X_orginal_y[index][0])    
            if check_list[index] != 'ok':
                print(f'{key:<10}: {(val-ref)/ref*100:>10.2f}% ({check_list[index]})')
            else:   
                print(f'{key:<10}: {(val-ref)/ref*100:>10.2f}%')

    Vds_range = np.arange(-3.5, 8.5 + 0.1, 0.1)

    # plot validation result
    # legend_values = np.linspace(1.5, 6.5, 6)  
    legend_values = np.linspace(1, 7, 7)  
    # create colormap
    cmap = plt.cm.winter
    norm = plt.Normalize(vmin=legend_values.min(), vmax=legend_values.max())

    fig, ax = plt.subplots(1, 3, figsize=(15, 8)) 
    RMSE = np.mean((X_pred_iv - X_orginal_iv) ** 2)
    MAE = np.mean(np.abs(X_pred_iv - X_orginal_iv))
    NMAE = MAE / np.mean(X_orginal_iv) *100
    for i in range(7):
        color = cmap(norm(legend_values[i]))
        ax[0].plot( Vds_range, X_orginal_iv[i, :], label=f"VGS={i+1} V", color = color)
        ax[1].plot( Vds_range, X_pred_iv[i, :], label=f"VGS={i+1} V", color = color)
        ax[2].plot( Vds_range, X_pred_iv[i, :] - X_orginal_iv[i, :], label=f"VGS={i+1} V", color = color)

    ax[0].legend()
    ax[0].grid(True)
    ax[0].set_title('original I-V (generated, with noise)')
    ax[0].set_xlabel("VDS (V)")
    ax[0].set_ylabel("IDS (A)")
    ax[1].grid(True)
    ax[1].legend()
    ax[1].set_title('predicted I-V (NN based)')
    ax[1].set_xlabel("VDS (V)")
    ax[1].set_ylabel("IDS (A)")
    ax[2].grid(True)
    ax[2].set_title(f'error of both I-V \n average MAE = {MAE:.3f} \n nMAE = {NMAE:.3f}%')
    ax[2].set_xlabel("VDS (V)")
    ax[2].set_ylabel("IDS (A)")

    # add color bar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("VGS (V)")


In [10]:
# necessary class and functions for ADS simulation

class PyADS():
    def __init__(self):
        self.HPEESOF_DIR = 'D:/ADS/install'
        self.HOME = 'D:/ADS/dir'
        self.cur_workspace_path = None
        self.workspace = None
        self.cur_library_name = None
        self.library = None
        self.cur_design_name = None
        self.design = None

    def create_and_open_an_empty_workspace(self, workspace_path: str):
    # example : workspace_path = "C:/ADS_Python_Tutorials/tutorial1_wrk"
    # Ensure there isn't already a workspace open
        if de.workspace_is_open():
            de.close_workspace()
    
        # Cannot create a workspace if the directory already exists
        if os.path.exists(workspace_path):
            raise RuntimeError(f"Workspace directory already exists: {workspace_path}")
    
        # Create the workspace
        workspace = de.create_workspace(workspace_path)
        # Open the workspace
        workspace.open()
        # Return the open workspace and close when it finished
        return workspace
    
    def create_a_library_and_add_it_to_the_workspace(self, workspace: de.Workspace, library_name: str) -> None:
        #assert workspace.path is not None
        # Libraries can only be added to an open workspace
        assert workspace.is_open
        # We'll create a library in the directory of the workspace
        library_path = workspace.path / library_name
        # Create the library
        de.create_new_library(library_name, library_path)
        # And add it to the workspace (update lib.defs)
        workspace.add_library(library_name, library_path, de.LibraryMode.SHARED)
        lib=workspace.open_library(library_name,library_path,de.LibraryMode.SHARED)
        return lib

    def schematic_simulation(self, workspace_path: str, library_name: str, design_name: str, instance_name: str, var_dict: dict, vgs_bias_param_sweep_name: str, vds_bias_param_sweep_name: str, vgs_bias_simulation_name: str, vds_bias_simulation_name: str) -> None:
        ''' Load Path and files, Edit the design variables, Simulate the design, and return the dataset '''

        # >> Load Path and files
        if not os.path.exists(workspace_path):
            raise RuntimeError(f"Workspace directory doesn't exist: {workspace_path}")
        if de.workspace_is_open():
            de.close_workspace()
        
        # Open the workspace
        # if (not self.workspace) or (self.cur_workspace_path != workspace_path):
        self.workspace = de.open_workspace(workspace_path)
        self.cur_workspace_path = workspace_path
        # Open the library
        # if (not self.library) or (self.cur_library_name != library_name):
        self.library = self.workspace.open_library(lib_name=library_name, mode=de.LibraryMode.SHARED)
        self.cur_library_name = library_name
        # Open the design
        # if (not self.design) or (self.cur_design_name != design_name):
        self.design = db.open_design((library_name, design_name, "schematic"), db.DesignMode.APPEND)
        self.cur_design_name = design_name

        # >> Edit the design variables
        # edit VAR
        v = self.design.get_instance(inst_name=instance_name)
        assert v.is_var_instance
        for var_name in var_dict:
            v.vars[var_name] = var_dict[var_name]
        # Save the design
        self.design.save_design()
        # Simulate the design
        output_dir = os.path.join(self.workspace.path, "output")
        netlist_file = os.path.join(output_dir, "data_gen.ckt")
        output_file =  os.path.join(output_dir, "data_gen.ckt.out")
        # create the simulation output directory
        util.safe_makedirs(output_dir)

        # >> Simulate and return the dataset
        ipython = getipython.get_ipython()
        if ipython is None:
            print("The remaining portion of the script must be run in an IPython environment. Exiting.")
            return
        # capture the netlist in a string
        netlist = self.design.generate_netlist()
        # access to the simulator object to run netlists
        simulator = ads.CircuitSimulator()
        # run the netlist, this will block output
        simulator.run_netlist(netlist, output_dir=output_dir, netlist_file=netlist_file, output_file=output_file)
        output_data = dataset.open(Path(os.path.join(output_dir, f"{design_name}.ds")))
        
        # >> return data in pandas DataFrame format
        # <class 'pandas.core.frame.DataFrame'>
        data_ids_vds = output_data[f'{vgs_bias_param_sweep_name}.{vgs_bias_simulation_name}.DC'].to_dataframe().reset_index()
        data_ids_vgs = output_data[f'aele_0.{vds_bias_param_sweep_name}.{vds_bias_simulation_name}'].to_dataframe().reset_index()
        return data_ids_vds, data_ids_vgs
    


    def dataset_reshape(self, pd_data_IV: pd.DataFrame, pd_data_gm: pd.DataFrame, IV_dimension: list, gm_dimension: list, var_dict: dict):
        ''' reshape the dataset into desired input matrix and output vector '''
        IV_row_count = IV_dimension[0] # Vgs
        IV_col_count = IV_dimension[1] # Vds
        gm_row_count = gm_dimension[0] # Vds
        gm_col_count = gm_dimension[1] # Vgs

        output_x_IV = np.empty((IV_row_count, IV_col_count),dtype=np.float64)
        output_x_gm = np.empty((gm_row_count, gm_col_count),dtype=np.float64)
        output_y = np.empty((len(var_dict), 1),dtype=np.float64)

        for row in range(IV_row_count):
            output_x_IV[row, :] = pd_data_IV.loc[pd_data_IV['VGS'] == (row + 1), 'IDS.i'].to_numpy()
        for col in range(gm_col_count):
            output_x_gm[:, col] = pd_data_gm.loc[pd_data_gm['VGS'] == (col + 1.5), 'gm'].to_numpy()
        for index, item in enumerate(var_dict):
            output_y[index, 0] = var_dict[item]

        return output_x_IV, output_x_gm, output_y
    

def param_random_generator(param_range: dict):
    ''' generate a random parameter set for the HEMT model '''
    # define the parameter range
    # param_range = {
    #     'VOFF': (-1.2, 2.6),
    #     'U0': (0, 2.2),
    #     'NS0ACCS': (1e15, 1e20),
    #     'NFACTOR': (0.1, 5),
    #     'ETA0': (0, 1),
    #     'VSAT': (5e4, 1e7),
    #     'VDSCALE': (0.5, 1e6),
    #     'CDSCD': (1e-5, 0.75),
    #     'LAMBDA': (0, 0.2),
    #     'MEXPACCD': (0.05, 12),
    #     'DELTA': (2, 100)
    # }
    # generate random parameters
    var_dict = {key: str(np.random.uniform(low=val[0], high=val[1])) for key, val in param_range.items()}
    return var_dict

def init_h5_file(h5_path, x_iv_shape, x_gm_shape, y_shape,
                 dtype_x=np.float64, dtype_y=np.float64):
    with h5py.File(h5_path, 'w') as f:
        # X: [num_samples, m, n]
        f.create_dataset(
            'X_iv',
            shape=(0, x_iv_shape[0], x_iv_shape[1]),
            maxshape=(None, x_iv_shape[0], x_iv_shape[1]),
            dtype=dtype_x
        )
        f.create_dataset(
            'X_gm',
            shape=(0, x_gm_shape[0], x_gm_shape[1]),
            maxshape=(None, x_gm_shape[0], x_gm_shape[1]),
            dtype=dtype_x
        )
        # Y: [num_samples, y_len]
        f.create_dataset(
            'Y',
            shape=(0, y_shape[0], 1),
            maxshape=(None, y_shape[0], 1),
            dtype=dtype_y
        )

def append_to_h5(h5_path, x_iv_new, x_gm_new, y_new):
    x_iv_new = np.asarray(x_iv_new, dtype=np.float64)
    x_gm_new = np.asarray(x_gm_new, dtype=np.float64)
    y_new = np.asarray(y_new, dtype=np.float64)

    # 确保 y_new 是二维 (batch_size, y_len)
    if y_new.ndim == 2:
        y_new = y_new.reshape(-1, 1)
    else:
        raise RuntimeError(f"y_new must be vector, but got shape {y_new.shape}")

    with h5py.File(h5_path, 'a') as f:
        ds_x_iv = f['X_iv']
        ds_x_gm = f['X_gm']
        ds_y = f['Y']

        cur_len = ds_x_iv.shape[0]
        new_len = cur_len + 1

        # 扩展
        ds_x_iv.resize(new_len, axis=0)
        ds_x_gm.resize(new_len, axis=0)
        ds_y.resize(new_len, axis=0)

        # 赋值
        ds_x_iv[cur_len:new_len, :, :] = x_iv_new
        ds_x_gm[cur_len:new_len, :, :] = x_gm_new
        ds_y[cur_len:new_len, :] = y_new


def singel_process_iteration_data_gen2h5(workspace_path: str, validate_dict: dict, library_name: str, design_name: str, instance_name: str, param_range: dict, vgs_bias_param_sweep_name: str, vds_bias_param_sweep_name: str, vgs_bias_simulation_name: str, vds_bias_simulation_name: str, save_path: str, iteration_num: int  = 1, process_id: int = 1, dtype_x=np.float64, dtype_y=np.float64):
    ''' generate dataset in single process iteration '''
    # create an instance of the PyADS class
    ads_ctrl = PyADS()
    X_iv_shape = [7,121]
    X_gm_shape = [121,6]
    Y_shape = [11,1]

    # init h5 file
    if not validate_dict:
        init_h5_file(f"{save_path}\\dataset_process_{process_id}.h5", X_iv_shape, X_gm_shape, Y_shape)
    else:
        init_h5_file(f"{save_path}\\validate.h5", X_iv_shape, X_gm_shape, Y_shape)

    for i in range(iteration_num):
        start_time = time.time()
        if validate_dict:
            var_dict = validate_dict
        else:
            var_dict = param_random_generator(param_range)
        pd_data_vgs_bias, pd_data_gm = ads_ctrl.schematic_simulation(
            workspace_path,
            library_name,
            design_name,
            instance_name,
            var_dict,
            vgs_bias_param_sweep_name,
            vds_bias_param_sweep_name,
            vgs_bias_simulation_name,
            vds_bias_simulation_name
        )
        X_iv, X_gm, y = ads_ctrl.dataset_reshape(pd_data_vgs_bias, pd_data_gm, X_iv_shape, X_gm_shape, var_dict)
        end_time = time.time()
        if not validate_dict:
            print(f' >> Process {process_id} :: Loop {i + 1}/{iteration_num} :: used time:', round(end_time - start_time, 2), 's')
        else:
            print(f'finish validation in ADS :: used time:', round(end_time - start_time, 2), 's')

        try:
            if not validate_dict:
                append_to_h5(f"{save_path}\\dataset_process_{process_id}.h5", X_iv, X_gm, y)
            else:
                append_to_h5(f"{save_path}\\validate.h5", X_iv, X_gm, y)
        except:
            print(f"【ERROR】Error appending data in process {process_id} at iteration {i + 1}.")
            continue

In [11]:
# For ADS
# DEFINE VARIABLES

workspace_path = "E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Simulation\\ADS\\ASM_HEMT1_wrk_1_Jia"
validate_dict = None
library_name = "IAF_pGaN_lib"
design_name = "gs66508bv1_Pytest_simple_paramset"
instance_name = "IV"
var_dict_default = {'VOFF':'1.785', 'U0':'0.424', 'NS0ACCS':'2e+17', 'NFACTOR':'1', 'ETA0':'0.06', 'VSAT':'8e+4', 'VDSCALE':'5', 'CDSCD':'0.1', 'LAMBDA':'0.01', 'MEXPACCD':'1.5', 'DELTA':'3'}
param_range = {
        'VOFF': (-1.2, 2.6),
        'U0': (0, 2.2),
        'NS0ACCS': (1e15, 1e20),
        'NFACTOR': (0.1, 5),
        'ETA0': (0, 1),
        'VSAT': (5e4, 1e7),
        'VDSCALE': (0.5, 1e6),
        'CDSCD': (1e-5, 0.75),
        'LAMBDA': (0, 0.2),
        'MEXPACCD': (0.05, 12),
        'DELTA': (2, 100)
    }
vgs_bias_param_sweep_name = 'Sweep_vgs'
vds_bias_param_sweep_name = 'Sweep_vds'
vgs_bias_simulation_name = 'DC1'
vds_bias_simulation_name = 'DC2'
iteration_num = 1000
process_id = 1
data_save_path = "E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\temp_generated"


# ------------------------------------------------------------------
# For NN model inference

python_path = r"D:/Miniconda/envs/DL/python.exe"  # model based python path
script_dir  = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training" # path of code

training_data_path   = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\training\\stage_x_ml_full_data.h5"  # path of training dataset

meas_like_val_set = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_2_ml_data.h5" 

meas_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\training\\meas_data.h5"

output_csv_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\temp_generated\\pred_7row.csv"



In [12]:
'''single model inference'''


# model_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\runs_test\\log_dataset_test\\2_4_0_1_normal_log"

model_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\temp_2\\version_2_1"


# basic params
inference_data_path = training_data_path
inference_data_index = "7" #  from 0 to 9999
# "None"    : for 1. single input with shape (2,6,121)
#                 2. multile input with shaple (N,2,6,121)
# str(int)  : for single input with shape (N,2,6,121)
single_input_mode = bool(True)
cvae_ena = True
cvae_mode = 'mean'
num_sampling = 20 # only used when cvae_mode = 'rand'

# Model inference
print(" >> Start model inference...")
var_dict = model_inference(
    python_path=python_path,
    script_dir=script_dir,
    code_file_name="asm_hemt_2stage_dnn.py" if not cvae_ena else "asm_hemt_cvae.py",
    model_path=model_path,
    inference_data_path=inference_data_path,
    inference_data_index=inference_data_index,
    output_csv_path=output_csv_path,
    single_input_mode=single_input_mode,
    cvae_ena=cvae_ena,
    cvae_mode=cvae_mode,
    num_sampling=num_sampling
)

# range correction
print(" >> Start output range correction...")
invalid_list = ['ok']*len(param_range)
for index,key in enumerate(param_range):
    (low,high) = param_range[key]
    if float(var_dict[key]) < low:
        var_dict[key] = str(low)
        invalid_list[index] = 'Underflow'
    elif float(var_dict[key]) > high:
        var_dict[key] = str(high)
        invalid_list[index] = 'Overflow'


# main function to run the data generation - single process
print(" >> Start validate in ADS...")
singel_process_iteration_data_gen2h5(
    workspace_path=workspace_path,
    validate_dict=var_dict,
    library_name=library_name,
    design_name=design_name,
    instance_name=instance_name,
    param_range=param_range,
    vgs_bias_param_sweep_name=vgs_bias_param_sweep_name,
    vds_bias_param_sweep_name=vds_bias_param_sweep_name,
    vgs_bias_simulation_name=vgs_bias_simulation_name,
    vds_bias_simulation_name=vds_bias_simulation_name,
    save_path=data_save_path)


# plot error between prediction and original I-V data
print(" >> Start plot error...")
plot_error(inference_data_path, data_save_path, inference_data_index, var_dict, invalid_list)

# {'VOFF':'1.785', 'U0':'0.424', 'NS0ACCS':'2e+17', 'NFACTOR':'1', 'ETA0':'0.06', 'VSAT':'8e+4', 'VDSCALE':'5', 'CDSCD':'0.1', 'LAMBDA':'0.01', 'MEXPACCD':'1.5', 'DELTA':'3'}

 >> Start model inference...
Using CUDA device 0: NVIDIA GeForce RTX 4060 Laptop GPU

--- Solution Statistics (for first input) ---
  VOFF     : mean=1.06  std=nan
  U0       : mean=-1.132  std=nan
  NS0ACCS  : mean=-0.702  std=nan
  NFACTOR  : mean=-0.001651  std=nan
  ETA0     : mean=-0.877  std=nan
  VSAT     : mean=0.4307  std=nan
  VDSCALE  : mean=-0.8742  std=nan
  CDSCD    : mean=-0.7956  std=nan
  LAMBDA   : mean=0.2813  std=nan
  MEXPACCD : mean=0.1187  std=nan
  DELTA    : mean=0.1267  std=nan

Saved all 1 sampled solutions to E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\temp_generated\\pred_7row.csv



KeyError: 'param'

In [ ]:
'''Ensemble Model Inference'''

model_path_prefix = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/runs_test/2stage_ensemble/version_"
ensemble_num = 2

# basic params
inference_data_path = meas_path
inference_data_index = "None" #  from 0 to 9999
# "None"    : for 1. single input with shape (2,6,121)
#                 2. multile input with shaple (N,2,6,121)
# str(int)  : for single input with shape (N,2,6,121)
single_input_mode = bool(True)


# Model inference
print(" >> Start ensemble model inference...")
var_dict = [dict()]*ensemble_num
var_dict_ensemble = {}
for i in range(ensemble_num):
    print(f" >> Start model inference {i+1}...")
    model_temp_path = f"{model_path_prefix}{i+1}"
    var_dict[i] = model_inference(
        python_path=python_path,
        script_dir=script_dir,
        code_file_name=code_file_name,
        model_path=model_temp_path,
        inference_data_path=inference_data_path,
        inference_data_index=inference_data_index,
        output_csv_path=output_csv_path,
        single_input_mode=single_input_mode
    )

for index,key in enumerate(param_range):
    var_dict_ensemble[key] = str(float(np.mean([float(var_dict[i][key]) for i in range(ensemble_num)])))

print(" >> Ensemble model inference results:")
for key in var_dict_ensemble:
    print(f"{key:<10}: {float(var_dict_ensemble[key]):>12.4e}")
print('\n')

# main function to run the data generation - single process
print(" >> Start validate in ADS for ensemble model...")
singel_process_iteration_data_gen2h5(
    workspace_path=workspace_path,
    validate_dict=var_dict_ensemble,
    library_name=library_name,
    design_name=design_name,
    instance_name=instance_name,
    param_range=param_range,
    vgs_bias_param_sweep_name=vgs_bias_param_sweep_name,
    vds_bias_param_sweep_name=vds_bias_param_sweep_name,
    vgs_bias_simulation_name=vgs_bias_simulation_name,
    vds_bias_simulation_name=vds_bias_simulation_name,
    save_path=data_save_path)


# plot error between prediction and original I-V data
print(" >> Start plot error for ensemble model...")
invalid_list = ['ok']*len(param_range)
plot_error(inference_data_path, data_save_path, inference_data_index, var_dict, invalid_list)

In [ ]:
"proxy model G inference -> asm-hemt model"

python_path = r"D:/Miniconda/envs/DL/python.exe"  # model based python path
script_dir  = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training" # path of code
code_file_name = "asm_hemt_2stage_dnn.py"


meas_like_val_set = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_2_ml_data.h5"  # path of meas like dataset

g_training_set = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_x_ml_full_data.h5" # path of training dataset

inference_data_path = g_training_set
meas_path = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/meas_data.h5"

output_npy_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\dataset\\temp_generated\\xhat.npy"

model_path = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\NN_training\\model\\runs_2stage_3rd_version\\proxy_g\\best_g_log_bigger"

inference_data_index = '8526'

proxy_model_inference(python_path = python_path, 
                     script_dir = script_dir, 
                     code_file_name = code_file_name, 
                     model_path = model_path, 
                     inference_data_path = inference_data_path, 
                     inference_data_index = inference_data_index,
                     output_npy_path = output_npy_path)

plot_proxy_error(inference_data_path, 
                 output_npy_path, 
                 inference_data_index)